In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import sys
sys.path.append('../src')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model, Input, layers
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, Conv1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
from bert import tokenization
import warnings 
warnings.filterwarnings(action='ignore')

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## BERT Tokenizer 로드

In [4]:
data_dir = Path('C:/Users/ekdms/dacon-author-classification')
feature_dir = Path('C:/Users/ekdms/build/feature')
val_dir = Path('C:/Users/ekdms/build/val')
tst_dir = Path('C:/Users/ekdms/build/tst')
sub_dir = Path('C:/Users/ekdms/build/sub')
dirs = [feature_dir, val_dir, tst_dir, sub_dir]
for d in dirs:
    os.makedirs(d, exist_ok=True)

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [5]:
algo_name = 'bert-attention5'
max_len = 250
feature_name = f'n{max_len}'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [6]:
bert_layer = hub.KerasLayer(module_url, trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

## 학습데이터 로드

In [7]:
train = pd.read_csv(trn_file, index_col=0)
test = pd.read_csv(tst_file, index_col=0)

## Preprocessing

In [8]:
# https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub
def bert_encode(texts, tokenizer, max_len=max_len):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [9]:
trn = bert_encode(train.text.values, tokenizer, max_len=max_len)
tst = bert_encode(test.text.values, tokenizer, max_len=max_len)
y = train['author'].values
print(trn[0].shape, tst[0].shape)

(54879, 250) (19617, 250)


## Training

In [10]:
vocab_size = 50000
maxlen = 250
embed_dim = 128
num_heads = 8  # Number of attention heads
ff_dim = 128
padding_type='post'

In [11]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [14]:
def get_model():
    model = Sequential([
        Embedding(vocab_size, embed_dim, input_length=maxlen),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64)),
        Dense(n_class, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    return model

In [16]:
p_val = np.zeros((trn[0].shape[0], n_class))
p_tst = np.zeros((tst[0].shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn[0], y), 1):
    print(f'training model for CV #{i}')
    clf = get_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf.fit([x[i_trn] for x in trn], 
            to_categorical(y[i_trn]),
            validation_data=([x[i_val] for x in trn], to_categorical(y[i_val])),
            epochs=10,
            batch_size=128,
           callbacks=[es])
    p_val[i_val, :] = clf.predict([x[i_val] for x in trn])
    p_tst += clf.predict(tst) / n_fold

    clear_session()
    gc.collect()

training model for CV #1
Epoch 1/10
343/343 [==============================] - 1691s 5s/step - loss: 0.9054 - val_loss: 0.6399
Epoch 2/10
343/343 [==============================] - 1857s 5s/step - loss: 0.4792 - val_loss: 0.5479
Epoch 3/10
343/343 [==============================] - 1828s 5s/step - loss: 0.3300 - val_loss: 0.5553
Epoch 4/10
343/343 [==============================] - 1862s 5s/step - loss: 0.2514 - val_loss: 0.6412
Epoch 5/10
343/343 [==============================] - 1842s 5s/step - loss: 0.2063 - val_loss: 0.6767
Epoch 00005: early stopping
training model for CV #2
Epoch 1/10
343/343 [==============================] - 2377s 7s/step - loss: 0.9190 - val_loss: 0.6775
Epoch 2/10
343/343 [==============================] - 2066s 6s/step - loss: 0.4935 - val_loss: 0.5677
Epoch 3/10
343/343 [==============================] - 2012s 6s/step - loss: 0.3404 - val_loss: 0.5811
Epoch 4/10
343/343 [==============================] - 1816s 5s/step - loss: 0.2610 - val_loss: 0.5945
Epoc

In [17]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Accuracy (CV):  79.8047%
Log Loss (CV):   0.5618


In [18]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [19]:
print(clf.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 128)          6400000   
_________________________________________________________________
bidirectional (Bidirectional (None, 250, 128)          98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 6,598,277
Trainable params: 6,598,277
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [21]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.0607,0.3648,0.3895,0.1795,0.0054
1,0.1328,0.7117,0.0109,0.0511,0.0934
2,0.9586,0.0172,0.0125,0.0073,0.0044
3,0.0009,0.0070,0.9855,0.0055,0.0012
4,0.8883,0.0197,0.0290,0.0418,0.0212


In [22]:
sub.to_csv(sub_file)